# Import libraries

In [1]:
import matplotlib.pyplot as plt 
from random import randint

# Helper classes

In [2]:
# output format: length either 250 or 5500 samples

class emg_cleaner:
    noise = []
    
    def __init__(self, address):
        with open(address, "r") as f:
            data = f.read()
        data = data.split("\n")
        data.pop()
        for i in range(len(data)):
            data[i] = data[i].split(";")
            data[i].pop()
            for j in range(len(data[i])):
                data[i][j] = float(data[i][j])
                data[i][j] = int(data[i][j])
        
        self.emg_channels = [[], [], [], [], [], [], [], []]
        for i in range(len(self.emg_channels)):
            for j in range(len(data)):
                self.emg_channels[i].append(data[j][i])
                
        self.timeline = [i for i in range(len(data))]
        
        
    def set_noise(self, noise_address):
        with open(noise_address, "r") as f:
            data = f.read()
        data = data.split("\n")
        data.pop()
        for i in range(len(data)):
            data[i] = data[i].split(";")
            data[i].pop()
            for j in range(len(data[i])):
                data[i][j] = float(data[i][j])
                data[i][j] = int(data[i][j])
        
        emg_cleaner.noise  = [[], [], [], [], [], [], [], []]
        for i in range(len(emg_cleaner.noise)):
            for j in range(len(data)):
                emg_cleaner.noise[i].append(data[j][i])
        
        
    def print_noise(self):
        print(emg_cleaner.noise[0])
    
    
    def save(self, address):
        with open(address, "w") as f:
            for i in range(len(self.emg_channels[0])):
                for j in range(len(self.emg_channels)):
                    f.write(str(self.emg_channels[j][i]))
                    f.write(";")
                f.write("\n")
        
        
    def mk_plot(self, channel, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.timeline, self.emg_channels[channel])
        axes.relim()
        axes.set_xlabel('X-Axis')
        axes.set_ylabel('Y-Axis')
        axes.set_title('1 channel')
        
        
    def mk_plots(self, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.timeline, self.emg_channels[0], 
                  self.timeline, self.emg_channels[1], 
                  self.timeline, self.emg_channels[2],
                  self.timeline, self.emg_channels[3],
                  self.timeline, self.emg_channels[4],
                  self.timeline, self.emg_channels[5],
                  self.timeline, self.emg_channels[6],
                  self.timeline, self.emg_channels[7])
        axes.set_xlabel('smaple')
        axes.set_ylabel('EMG')
        axes.set_title('All channels') 
        
        
    def delete_points_from_start(self, points):
        for i in range(points):
            for j in range(len(self.emg_channels)):
                self.emg_channels[j].pop(0)                
        self.timeline = [i for i in range(len(self.timeline) - points)]
        
        
    def delete_points_from_end(self, points):
        for i in range(points):
            for j in range(len(self.emg_channels)):
                self.emg_channels[j].pop()
        self.timeline = [i for i in range(len(self.timeline) - points)]
    
    
    def add_points_to_start(self, points):
        random_start = randint(0, len(emg_cleaner.noise[0]) - 1 - points)
        for i in range(points):
            for j in range(len(self.emg_channels)):
                self.emg_channels[j].insert(i, emg_cleaner.noise[j][random_start+i])
        self.timeline = [i for i in range(len(self.timeline) + points)]
    
    
    def replace_points_in_middle(self, start, end):
        random_start = randint(0, len(emg_cleaner.noise[0]) - 2 - end + start)
        for i in range(start, end + 1):
            for j in range(len(self.emg_channels)):
                
                self.emg_channels[j][i] = emg_cleaner.noise[j][random_start+i]

    
    
    def add_points_to_end(self, points):
        random_start = randint(0, len(emg_cleaner.noise[0]) - 1 - points)
        for i in range(points):
            for j in range(len(self.emg_channels)):
                self.emg_channels[j].append(emg_cleaner.noise[j][random_start+i])
        self.timeline = [i for i in range(len(self.timeline) + points)]

In [3]:
# This class allows to classify file

class class_editor:
    def __init__(self, address):
        with open(address, "r") as f:
            data = f.read()
        data = data.split("\n")
        data.pop()
        for i in range(len(data)):
            data[i] = data[i].split(";")
            data[i].pop()
            for j in range(len(data[i])):
                data[i][j] = float(data[i][j])
                data[i][j] = int(data[i][j])
        
        self.emg_channels = [[], [], [], [], [], [], [], []]
        for i in range(len(self.emg_channels)):
            for j in range(len(data)):
                self.emg_channels[i].append(data[j][i])
                
        self.timeline = [i for i in range(len(data))]
        self.classification = [0 for i in range(len(data))]
        self.classplot = [-50 for i in range(len(data))]
        
        
    def set_classification(self, edges, class_name):
        self.classification = [0 for i in range(len(self.timeline))]
        self.classplot = [-50 for i in range(len(self.timeline))]
        for i in range(len(edges)):
            for j in range(edges[i][0], edges[i][1] + 1):
                self.classification[j] = class_name
                if class_name == 0: self.classplot[j] = -50
                else: self.classplot[j] = 50
                    
    
    def save(self, address):
        with open(address, "w") as f:
            for i in range(len(self.emg_channels[0])):
                for j in range(len(self.emg_channels)):
                    f.write(str(self.emg_channels[j][i]))
                    f.write(";")
                f.write(str(self.classification[i]))
                f.write(";")
                f.write("\n")
   

    def mk_plot(self, channel, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.timeline, self.emg_channels[channel])
        axes.relim()
        axes.set_xlabel('X-Axis')
        axes.set_ylabel('Y-Axis')
        axes.set_title('1 channel')
        
        
    def mk_plots(self, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.timeline, self.emg_channels[0], 
                  self.timeline, self.emg_channels[1], 
                  self.timeline, self.emg_channels[2],
                  self.timeline, self.emg_channels[3],
                  self.timeline, self.emg_channels[4],
                  self.timeline, self.emg_channels[5],
                  self.timeline, self.emg_channels[6],
                  self.timeline, self.emg_channels[7],
                  self.timeline, self.classplot)
        axes.set_xlabel('smaple')
        axes.set_ylabel('EMG')
        axes.set_title('All channels') 

In [4]:
# This class allows to produce file output.csv file containing 7 features
# for 8 channels of EMG signal with timestamps. Format of output data is
#
#   [[[MAV1,  [...,  [CHP1,      CLASS0,
#      MAV1,   ...,   CHP1,      CLASS1,
#      ...................,      ......,
#      ...................,      ......,
#      ...................,      ......,
#      MAV1],  ...],  CHP1]],    CLASSN,
#
# (no white space included). Input files should have the following format:

class converter2:
    def __init__(self, l, L, H, address):
        self.l = l
        self.L = L
        self.H = H
        
        with open(address, "r") as f:
            data = f.read()
        data = data.split("\n")
        data.pop()
        for i in range(len(data)):
            data[i] = data[i].split(";")
            data[i].pop()
            for j in range(len(data[i])):
                data[i][j] = float(data[i][j])
        
        # emg fields init
        self.emg_channels = [[], [], [], [], [], [], [], []]
        self.emg_class = []
        self.emg_classplot = []
        self.emg_timeline = [i for i in range(len(data))]
        
        # fill emg channels
        for i in range(len(self.emg_channels)):
            for j in range(len(data)):
                self.emg_channels[i].append(data[j][i])

        # fill eg classes        
        for i in range(len(data)):
            self.emg_class.append(data[i][8])
            if data[i][8] == 0: self.emg_classplot.append(-50)
            else: self.emg_classplot.append(50)
        
        
        # features fields init
        self.feature_channels = [[[], [], [], [], [], [], []], 
                                 [[], [], [], [], [], [], []], 
                                 [[], [], [], [], [], [], []],
                                 [[], [], [], [], [], [], []],
                                 [[], [], [], [], [], [], []],
                                 [[], [], [], [], [], [], []],
                                 [[], [], [], [], [], [], []],
                                 [[], [], [], [], [], [], []]]
        self.feature_class = []
        self.feature_classplot = []
        self.feature_timeline = []

        
    def save(self, address):
        with open(address, "w") as f:
            for k in range(len(self.feature_channels[0][0])):
                for i in range(len(self.feature_channels)):
                    for j in range(len(self.feature_channels[i])):
                        f.write(str(self.feature_channels[i][j][k]))
                        f.write(";")
                f.write(str(self.feature_class[k]))
                f.write(";")
                f.write("\n")
        
        
    def process_file(self, custom_class = 1):
        self.process_timeline()
        self.process_classes(custom_class = custom_class)
        self.process_channels()
        
        
    def process_timeline(self):
        # calculate number of rows in output
        windows_num = int (int(len(self.emg_channels[0])) / int(self.l) - self.L + 1)
        
        for i in range(windows_num):
            self.feature_timeline.append(i * self.l + self.l * self.L / 2)
        
    
    def process_classes(self, custom_class = 1):
        # calculate number of rows in output
        windows_num = int (int(len(self.emg_channels[0])) / int(self.l) - self.L + 1)
        
        for i in range(windows_num):
            classes = [0, 0]
            for j in range(i * self.l, (i + self.L) * self.l):
                if self.emg_class[j] == 0: classes[1] += 1
                else: classes[0] += 1
            if classes[0] > 3 * classes[1]: 
                self.feature_class.append(custom_class)
                self.feature_classplot.append(0.5)
            else: 
                self.feature_class.append(0)
                self.feature_classplot.append(-0.5)
        
    
    def process_channels(self):
        for i in range(len(self.feature_channels)):
            self.process_channel(i)
    
    
    def process_channel(self, num):
        # calculate number of rows in output
        windows_num = int (int(len(self.emg_channels[0])) / int(self.l) - self.L + 1)
        
        for i in range(windows_num):       
            f = self.process_window(self.emg_channels[num][i * self.l:(i + self.L) * self.l - 1])
            for i in range(len(f)):
                self.feature_channels[num][i].append(f[i])

        
    def process_window(self, window):
        # normilize input data
        for i in range(len(window)):
            window[i] = float(window[i])/128

        # calculate features
        f = []
        f.append(self.get_MAV(window))
        f.append(self.get_RMS(window))
        f.append(self.get_SSC(window))
        f.append(self.get_WL(window))
        f.append(self.get_AHP(window))
        f.append(self.get_MHP(window))
        f.append(self.get_CHP(window))
        
        return f
     
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns their mean value
    def get_MAV(self, w):
        
        sum1 = 0
        for x in range(0,len(w)):
            sum1 = sum1+w[x]
            
        return sum1/len(w)
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns their mean power
    def get_RMS(self, w):
        
        sum1 = 0 
        for x in range(0,len(w)):
            sum1 = sum1 + w[x]**2
        
        return (sum1/len(w))**(0.5)
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns frequency parameter
    def get_SSC(self, w):
        
        sum1 = 0
        for x in range(1, len(w)-1):
            sum1 = sum1 + abs((w[x]-w[x-1])*(w[x]-w[x+1]))
        
        return sum1
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns cumulative wavelength
    def get_WL(self, w):
        sum1 = 0
        for x in range(1, len(w)):
            sum1 = sum1 + abs(w[x]-w[x-1])
        
        return sum1
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns parameter AHP
    def get_AHP(self, w):
        sum1 = 0
        for x in range(0, len(w)):
            sum1 = sum1 + w[x]**2
        
        return 1/(len(w)-1) * sum1
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns parameter MHP
    def get_MHP(self, w):
        
        # AHP w/ s
        sum1 = 0
        for x in range(0, len(w)):
            sum1 = sum1 + w[x]**2
        
        # AHP w/ s'
        sum2 = 0
        for x in range(0, len(w)-1):
            sum2 = sum2 + (w[x+1] - w[x])**2
            
        return (sum2/sum1)**(0.5)
    
    
    # this method gets a list "w" consisting of l * L samples from
    # float[-1, 1] for 1 EMG channel and returns parameter CHP
    def get_CHP(self, w):
        CHP = 0
        return CHP
    

    def mk_plots(self, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.feature_timeline, self.feature_channels[0][1], 
                  self.feature_timeline, self.feature_channels[1][1], 
                  self.feature_timeline, self.feature_channels[2][1],
                  self.feature_timeline, self.feature_channels[3][1],
                  self.feature_timeline, self.feature_channels[4][1],
                  self.feature_timeline, self.feature_channels[5][1],
                  self.feature_timeline, self.feature_channels[6][1],
                  self.feature_timeline, self.feature_channels[7][1],
                  self.feature_timeline, self.feature_classplot)
        axes.set_xlabel('smaple')
        axes.set_ylabel('EMG')
        axes.set_title('All channels') 
    
    
    def mk_emg_plots(self, x_segment, y_scale):       
        figure, axes = plt.subplots(figsize = (15,10))
        axes.set(xlim=(500*x_segment, 500*(x_segment+1)), ylim=(-y_scale, y_scale))
        axes.plot(self.emg_timeline, self.emg_channels[0], 
                  self.emg_timeline, self.emg_channels[1], 
                  self.emg_timeline, self.emg_channels[2],
                  self.emg_timeline, self.emg_channels[3],
                  self.emg_timeline, self.emg_channels[4],
                  self.emg_timeline, self.emg_channels[5],
                  self.emg_timeline, self.emg_channels[6],
                  self.emg_timeline, self.emg_channels[7],
                  self.emg_timeline, self.emg_classplot)
        axes.set_xlabel('smaple')
        axes.set_ylabel('EMG')
        axes.set_title('All channels') 

# Addresses

In [5]:
directory = {
    "noise": "data/raw_data/noise/noise_reference 1.txt",
    "noise 2": "data/raw_data/noise/noise_reference 2.txt",
    "noise 3": "data/raw_data/noise/noise_reference 2.txt",
    "noise taps 1": "data/raw_data/noise/taps_noise 1.txt",
    "noise taps 2": "data/raw_data/noise/taps_noise 2.txt",
    "noise taps 3": "data/raw_data/noise/taps_noise 3.txt",
    "noise taps 4": "data/raw_data/noise/taps_noise 4.txt",

    
    # features 4 data
    "A_features4": "data/features4/taps 2/A/",
    "D_alt_features4": "data/features4/taps 2/D alt/",
    "E_alt_features4": "data/features4/taps 2/E alt/",
    "R_alt_features4": "data/features4/taps 2/R alt/", 
    "S_features4": "data/features4/taps 2/S/",
    "F_features4": "data/features4/taps 2/F/",    
    "F_alt_features4": "data/features4/taps 2/F alt/",

        
    # features 3 data
    "A_features3": "data/features3/taps 2/A/",
    "D_alt_features3": "data/features3/taps 2/D alt/",
    "S_features3": "data/features3/taps 2/S/",
    "F_features3": "data/features3/taps 2/F/",    

    
    # features 2 data
    "A_features2": "data/features2/taps 2/A/",
    "D_alt_features2": "data/features2/taps 2/D alt/",
    "S_features2": "data/features2/taps 2/S/",
    "F_features2": "data/features2/taps 2/F/",
    
    
    # features 1 data
    "index_finger_features1": "data/features1/taps/index_finger/",
    "middle_finger_features1": "data/features1/taps/middle_finger/",
    "ring_finger_features1": "data/features1/taps/ring_finger/",
    "little_finger_features1": "data/features1/taps/little_finger/",
    
    "hand_up_features1": "data/features1/hand_movement/up/",
    "hand_down_features1": "data/features1/hand_movement/down/",
    "hand_left_features1": "data/features1/hand_movement/left/",
    "hand_right_features1": "data/features1/hand_movement/right/",
    
    "A_features1": "data/features1/taps 2/A/",
    "D_features1": "data/features1/taps 2/D/",
    "D_alt_features1": "data/features1/taps 2/D alt/",
    "E_features1": "data/features1/taps 2/E/",
    "E_new_features1": "data/features1/taps 2/E new/",
    "E_alt_features1": "data/features1/taps 2/E alt/",
    "F_features1": "data/features1/taps 2/F/",
    "F_alt_features1": "data/features1/taps 2/F alt/",
    "R_features1": "data/features1/taps 2/R/",
    "R_new_features1": "data/features1/taps 2/R new/",
    "R_alt_features1": "data/features1/taps 2/R alt/",
    "S_features1": "data/features1/taps 2/S/",
    "W_features1": "data/features1/taps 2/W/",
    "W_new_features1": "data/features1/taps 2/W new/",

    "Michael_A_features": "data/features1/Michael_taps/A/",
    "Michael_D_features": "data/features1/Michael_taps/D/",
    "Michael_E_features": "data/features1/Michael_taps/E/",
    "Michael_F_features": "data/features1/Michael_taps/F/",
    "Michael_R_features": "data/features1/Michael_taps/R/",
    "Michael_S_features": "data/features1/Michael_taps/S/",
    "Michael_W_features": "data/features1/Michael_taps/W/",
    
    
    # classified data
    "index_finger_classified": "data/classified_data/taps/index_finger/",
    "middle_finger_classified": "data/classified_data/taps/middle_finger/",
    "ring_finger_classified": "data/classified_data/taps/ring_finger/",
    "little_finger_classified": "data/classified_data/taps/little_finger/",
    
    "hand_up_classified": "data/classified_data/hand_movement/up/",
    "hand_down_classified": "data/classified_data/hand_movement/down/",
    "hand_left_classified": "data/classified_data/hand_movement/left/",
    "hand_right_classified": "data/classified_data/hand_movement/right/",
    
    "A_classified": "data/classified_data/taps 2/A/",
    "D_classified": "data/classified_data/taps 2/D/",
    "D_alt_classified": "data/classified_data/taps 2/D alt/",
    "E_classified": "data/classified_data/taps 2/E/",
    "E_new_classified": "data/classified_data/taps 2/E new/",
    "E_alt_classified": "data/classified_data/taps 2/E alt/",
    "F_classified": "data/classified_data/taps 2/F/",
    "F_alt_classified": "data/classified_data/taps 2/F alt/",
    "R_classified": "data/classified_data/taps 2/R/",
    "R_new_classified": "data/classified_data/taps 2/R new/",
    "R_alt_classified": "data/classified_data/taps 2/R alt/",
    "S_classified": "data/classified_data/taps 2/S/",
    "W_classified": "data/classified_data/taps 2/W/",
    "W_new_classified": "data/classified_data/taps 2/W new/",

    "Michael_A_classified": "data/classified_data/Michael_taps/A/",
    "Michael_D_classified": "data/classified_data/Michael_taps/D/",
    "Michael_E_classified": "data/classified_data/Michael_taps/E/",
    "Michael_F_classified": "data/classified_data/Michael_taps/F/",
    "Michael_R_classified": "data/classified_data/Michael_taps/R/",
    "Michael_S_classified": "data/classified_data/Michael_taps/S/",
    "Michael_W_classified": "data/classified_data/Michael_taps/W/",
    
    
    # clean data
    "index_finger_clean": "data/cleaned_data/taps/index_finger/",
    "middle_finger_clean": "data/cleaned_data/taps/middle_finger/",
    "ring_finger_clean": "data/cleaned_data/taps/ring_finger/",
    "little_finger_clean": "data/cleaned_data/taps/little_finger/",
    
    "hand_up_clean": "data/cleaned_data/hand_movement/up/",
    "hand_down_clean": "data/cleaned_data/hand_movement/down/",
    "hand_left_clean": "data/cleaned_data/hand_movement/left/",
    "hand_right_clean": "data/cleaned_data/hand_movement/right/",

    "A_clean": "data/cleaned_data/taps 2/A/",
    "D_clean": "data/cleaned_data/taps 2/D/",
    "D_alt_clean": "data/cleaned_data/taps 2/D alt/",
    "E_clean": "data/cleaned_data/taps 2/E/",
    "E_new_clean": "data/cleaned_data/taps 2/E new/",
    "E_alt_clean": "data/cleaned_data/taps 2/E alt/",
    "F_clean": "data/cleaned_data/taps 2/F/",
    "F_alt_clean": "data/cleaned_data/taps 2/F alt/",
    "R_clean": "data/cleaned_data/taps 2/R/",
    "R_new_clean": "data/cleaned_data/taps 2/R new/",
    "R_alt_clean": "data/clean_data/taps 2/R alt/",
    "S_clean": "data/cleaned_data/taps 2/S/",
    "W_clean": "data/cleaned_data/taps 2/W/",
    "W_new_clean": "data/cleaned_data/taps 2/W new/",
    
    "Michael_A_clean": "data/cleaned_data/Michael_taps/A/",
    "Michael_D_clean": "data/cleaned_data/Michael_taps/D/",
    "Michael_E_clean": "data/cleaned_data/Michael_taps/E/",
    "Michael_F_clean": "data/cleaned_data/Michael_taps/F/",
    "Michael_R_clean": "data/cleaned_data/Michael_taps/R/",
    "Michael_S_clean": "data/cleaned_data/Michael_taps/S/",
    "Michael_W_clean": "data/cleaned_data/Michael_taps/W/",


    # raw data    
    "index_finger_raw": "data/raw_data/taps/index_finger/",
    "middle_finger_raw": "data/raw_data/taps/middle_finger/",
    "ring_finger_raw": "data/raw_data/taps/ring_finger/",
    "little_finger_raw": "data/raw_data/taps/little_finger/",
    
    "hand_up_raw": "data/raw_data/hand_movement/up/",
    "hand_down_raw": "data/raw_data/hand_movement/down/",
    "hand_left_raw": "data/raw_data/hand_movement/left/",
    "hand_right_raw": "data/raw_data/hand_movement/right/",
    
    "A_raw": "data/raw_data/taps 2/A/",
    "D_raw": "data/raw_data/taps 2/D/",
    "D_alt_raw": "data/raw_data/taps 2/D alt/",
    "E_raw": "data/raw_data/taps 2/E/",
    "E_new_raw": "data/raw_data/taps 2/E new/",
    "E_alt_raw": "data/raw_data/taps 2/E alt/",
    "F_raw": "data/raw_data/taps 2/F/",
    "F_alt_raw": "data/raw_data/taps 2/F alt/",
    "R_raw": "data/raw_data/taps 2/R/",
    "R_new_raw": "data/raw_data/taps 2/R new/",
    "R_alt_raw": "data/raw_data/taps 2/R alt/",
    "S_raw": "data/raw_data/taps 2/S/",
    "W_raw": "data/raw_data/taps 2/W/",
    "W_new_raw": "data/raw_data/taps 2/W new/",
    
    "Michael_A_raw": "data/raw_data/Michael_taps/A/",
    "Michael_D_raw": "data/raw_data/Michael_taps/D/",
    "Michael_E_raw": "data/raw_data/Michael_taps/E/",
    "Michael_F_raw": "data/raw_data/Michael_taps/F/",
    "Michael_R_raw": "data/raw_data/Michael_taps/R/",
    "Michael_S_raw": "data/raw_data/Michael_taps/S/",
    "Michael_W_raw": "data/raw_data/Michael_taps/W/"
}

# Preload data

In [ ]:
mcleaner = emg_cleaner(directory["little_finger_raw"]
                       + "set1, 1 gestures, 1 repeats, not classified, little finger keyboard press.txt")
mcleaner.set_noise(directory["noise taps 3"])

# Workspace

## Clean

In [ ]:
set_number = 9
input_file="A"+str(set_number)+".txt"
output_file="A"+str(set_number)+", clean.txt"

mcleaner = emg_cleaner(directory["A_raw"] + input_file)


mcleaner.set_noise(directory["noise taps 3"])


#mcleaner.delete_points_from_start(40)
#mcleaner.delete_points_from_end(28)
#mcleaner.add_points_to_start(29)
#mcleaner.add_points_to_end(29)
#print(len(mcleaner.timeline))
#mcleaner.replace_points_in_middle(1030, 1100)


print(len(mcleaner.timeline))
#mcleaner.delete_points_from_start(2)
#mcleaner.delete_points_from_end(12)
#mcleaner.add_points_to_start(22)
#mcleaner.add_points_to_end(12)
print(len(mcleaner.timeline))

mcleaner.mk_plots(0, 40)
mcleaner.save(directory["A_clean"]  + output_file)

## Classify

In [ ]:
set_number = 60

input_file="F"+str(set_number)+" alt.txt"
output_file="F"+str(set_number)+" alt, classified.txt"

mclass_editor = class_editor(directory["F_alt_raw"] + input_file)

print(len(mclass_editor.timeline))

mclass_editor.set_classification([(0, 70)], 1)
mclass_editor.mk_plots(0, 55)
mclass_editor.save(directory["F_alt_classified"] + output_file)

## Extract features

In [ ]:
#for i in range(1, 61):
#    if not (i == 5):
    set_number = i

    input_file="F"+str(set_number)+", classified.txt"
    output_file="F"+str(set_number)+", features.txt"

    mconverter = converter2(2, 40, 200, "data/classified_data/noise/taps_noise 4.txt")  

    mconverter.process_file(custom_class=1)
    mconverter.save("data/features4/noise/taps_noise 4.txt")

## Print features

In [ ]:
mconverter.mk_plots(0, 0.75)

# Reference

$l = 10ms$ ms, or $l = 2$ samples (for hand)

$L = 400$ ms, or $L = 80$ samples, or $L = 40$ segments (for hand)

classes:
1. Hand up
2. Hand down
3. Hand left
4. Hand right

5. Index finger tap